In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
import os
import logging

from actioncard import Chapel, Moat, Village, Militia, Moneylender, Smithy, CouncilRoom, Laboratory, Market, Witch
from aiutils import load, save
from config import GameConfig
from constants import ACTION, BUY
from enums import StartingSplit
from env import DefaultEnvironment, FullEnvironment
from player import load_players
from simulations import simulate

In [5]:
default_kingdom = [Chapel, Moat, Village, Militia, Moneylender, Smithy, CouncilRoom, Laboratory, Market, Witch]

In [6]:
project_root = '/home/justiny/Documents/Projects/PyDominion'
# project_root = '/home/justiny/Princeton/PyDominion'
model_dir = os.path.join(project_root, 'models')

In [7]:
logger = logging.getLogger()
logger.setLevel(logging.ERROR)
# logger.setLevel(BUY)
# logger.setLevel(ACTION)

## Rule-Based Bots ##

In [90]:
config = GameConfig(splits=[StartingSplit.StartingRandomSplit, StartingSplit.StartingRandomSplit], sandbox=True, num_players=2, must_include=[])
players = load_players(['BM', 'BM'], None, map_location='cpu', train=False)
agent = players[0]
env = FullEnvironment(config, players, logger=logger)

In [91]:
n = 10000

In [92]:
sim_stats = simulate(env, n, None, turn_log=True, action_log=True)

100%|██████████| 10000/10000 [03:08<00:00, 53.01it/s]
===SUMMARY===
{0: 5716, 1: 5574, 'ProvinceWins': 10000, 'Ties': 1290}


In [95]:
data_path = os.path.join(project_root, 'data', 'bm-bm-10k-stats')

In [96]:
save(data_path, sim_stats)

## Tree Simulations ##

In [15]:
from mcts import GameTree

In [46]:
# tree_name = 'full-paper-tree-uniform'
tree_name = 'r-r-mcts-bm-full-100k-srat-uniform-paper'
tree_path = os.path.join(model_dir, tree_name)
rollout_name = 'default-mlog-full-paper'
rollout_path = os.path.join(model_dir, rollout_name)

In [38]:
# tree_name = 'default-sandbox-uct-paper'
# tree_name = 'default-sandbox-uct-paper'
# tree_name = 'r-r-mcts-bm-hot-start-sandbox-100k-srat-uniform'
rollout_name = 'default-mlog-sandbox-paper'
tree_path = os.path.join(model_dir, tree_name)
rollout_path = os.path.join(model_dir, rollout_name)

In [47]:
tree = GameTree.load(tree_path, train=False, _skip_level=False, selection='max')

In [36]:
tree.selection = 'robust'
tree.selection

'robust'

In [13]:
tree.skip_level = True

In [56]:
# config = GameConfig(splits=[StartingSplit.Starting25Split, StartingSplit.Starting25Split], sandbox=True, num_players=2)
# splits = [StartingSplit.Starting34Split, StartingSplit.Starting25Split]
splits = [StartingSplit.StartingRandomSplit, StartingSplit.StartingRandomSplit]
# config = GameConfig(splits=splits, sandbox=True, num_players=2, must_include=[])
config = GameConfig(splits=splits, sandbox=False, must_include=default_kingdom)
players = load_players(['UCT', 'BM'], [rollout_path], tree=tree, rollout_type='mlog', train=False, use_tree=True)
# agent = load_players(['UCT'], [rollout_path], tree=tree, train=False, rollout_type='mlog', use_tree=True)[0]
# players = [agent, agent]

env = DefaultEnvironment(config, players, logger=logger)

In [20]:
logger.setLevel(logging.ERROR)
# logger.setLevel(BUY)

In [57]:
sim_data = simulate(env, 1000, tree, turn_log=False, action_log=False, card_log=False)
# sim_data = simulate(env, 1, tree, turn_log=False, action_log=False, card_log=False)

100%|██████████| 1000/1000 [00:25<00:00, 39.90it/s]
===SUMMARY===
{0: 956, 1: 51, 'ProvinceWins': 996, 'Ties': 7}


In [11]:
data_dir = os.path.join(project_root, 'data')
data_name ='ucthot25-bm25-1k-data'
data_path = os.path.join(data_dir, data_name)

In [12]:
save(data_path, sim_data)

In [23]:
tree._root.children

[Parent: None << (n: 8401, v: 5124.0, Q: 0.610 c: None) >> Children: [(0, 'Copper', '2018', '0.365'), (1, 'Estate', '1233', '0.328'), (2, 'None', '5150', '0.415')],
 Parent: None << (n: 41384, v: 18533.0, Q: 0.448 c: None) >> Children: [(0, 'Copper', '1117', '0.397'), (1, 'Estate', '611', '0.329'), (2, 'Silver', '37911', '0.566'), (3, 'None', '1745', '0.435')],
 Parent: None << (n: 41786, v: 18750.5, Q: 0.449 c: None) >> Children: [(0, 'Copper', '1471', '0.418'), (1, 'Estate', '785', '0.356'), (2, 'Silver', '37037', '0.567'), (3, 'None', '2493', '0.457')],
 Parent: None << (n: 8429, v: 4869.0, Q: 0.578 c: None) >> Children: [(0, 'Copper', '1075', '0.369'), (1, 'Estate', '360', '0.235'), (2, 'Duchy', '764', '0.334'), (3, 'Silver', '5382', '0.470'), (4, 'None', '848', '0.346')]]